# Exercises day 03

In [92]:
import numpy as np
import matplotlib.pyplot as plt
import scipy


def compute_confidence_interval_sample(data, confidence=0.95):
    N = len(data)
    alpha = 1- confidence
    #conf_mean = [data.mean - confidence * data.std()/np.sqrt(N), data.mean + confidence * data.std()/np.sqrt(N)]
    conf_mean = [data.mean + data.std()/np.sqrt(N) * scipy.stats.t.cdf(alpha/2, N - 1), data.mean + data.std()/np.sqrt(N) * scipy.stats.t.cdf(1-alpha/2, N - 1)]
    return conf_mean

## Ex01
The event for the blocking system is has the possible following event:

* Arrival of a customer 
* Customer is served
* Customer is blocked, as all $m$ service unints is occupied.


In [93]:
# Define blocking system
m = 10              # service units
service_mean = 8    # 1/mean service time
arrival_mean = 1    # mean arrival time      


Class for service units

In [94]:
class Event:
    def __init__(self, event_type : str, time : float, ) -> None:
        if event_type.lower() not in ['arrival', 'departure']:
            raise ValueError('event_type must be either arrival or departure')
        self.event_type = event_type
        self.time = time
       

In [95]:
class ServiceUnit:
    # def __init__(self, occupied : bool = False, service_time: float  = 0) -> None:
    def __init__(self, occupied : bool = False) -> None:
        self.occupied = occupied
        # self.service_time = service_time

   
    #def update_service_time(self, lam : float) -> None:
    #    self.service_time = numpy.random.exponential(lam)# + time

    def free_unit(self) -> None:
        self.occupied = False

    def take_unit(self) -> None:
        self.occupied = True 
         
    def __str__(self) -> str:
        return f"Occupied: {self.occupied}, Service Time: {self.service_time}"   



        

In [96]:
def sample_arrival() -> float:
    """
        
    """
    return np.random.exponential(arrival_mean) # NOTE: check if is it not 1/arrial_mean

def sample_service_time() -> float:
    """
        
    """
    return np.random.exponential(service_mean) 


def check_available_service(service_units : list[bool]) -> tuple[int, bool]:
    """
    
    """
    for indx, unit_occupied in enumerate(service_units):
        if not unit_occupied:
            return indx, True
        
    return None, False


def apend_event(event_list : list[Event], event_to_append : Event) -> list[Event]:
    """

    """
    for indx, event in enumerate(event_list):
        
        if event.time > event_to_append.time:
            event_list.insert(indx, event_to_append)
            return event_list
        
    event_list.append(event_to_append)
    return event_list
    

Simulation:

In [ ]:
def blocking_simulation(simulation_runs : int = 10, m : int = 10, N : int = 10000) -> list[float]:
    blocked_probabilities = []
    for _ in range(simulation_runs):
        custmer_count = 0
        global_time = 0
        event_counter = 0
        block_count = 0

        # lists
        event_list = []
        service_units_status = [False for _ in range(m)]

        # First arrival
        first_arrival = sample_arrival()
        event_list.append(Event('arrival', global_time + first_arrival))
        global_time += first_arrival
        event_list.append(Event('departure', global_time + sample_service_time()))
        service_units_status[0] = True # <-- unit 1 is occupied

        while custmer_count < N:
            
            current_event = event_list[event_counter]

            # Increment global time
            global_time = current_event.time

            if current_event.event_type == 'arrival':
                custmer_count += 1

                # Check for free service units
                indx, available = check_available_service(service_units_status)
                
                if available:
                    # Insert departure event and depend to eventlist
                    departure_event = Event('departure', global_time + sample_service_time())
                    event_list = apend_event(event_list, departure_event)

                    # Take service unit
                    service_units_status[indx] = True # <-- unit indx is occupied

                if not available:
                    # Costumer blocked
                    block_count += 1
                
                # insert time for next arrival
                arrival_event = Event('arrival', global_time + sample_arrival())
                event_list = apend_event(event_list, arrival_event)

            elif current_event.event_type == 'departure': 
                # Free the service unit for the current departure event
                for indx, unit_occupied in enumerate(service_units_status):
                    if unit_occupied:
                        service_units_status[indx] = False # <-- unit indx is free
                        
            # increment event counter
            event_counter += 1
        
        blocked_probabilities.append(block_count / N)
    
    return blocked_probabilities

Exact solution

In [103]:
A = arrival_mean *service_mean

B = (A**m/np.math.factorial(m))/np.sum([A**i/np.math.factorial(i) for i in range(m)])
print(f"Exact blocking probability: {B}")

Exact blocking probability: 0.13851266214160918


In [ ]:
# NOTE: Maybe use burn in period...?

# Init
N = 10**4
custmer_count = 0
global_time = 0
event_counter = 0
block_count = 0

# lists
event_list = []
ServiceUnits = [ServiceUnit() for _ in range(m)]

# sanity check
arrival_times = []
departure_times = []

# First arrival
first_arrival = sample_arrival()
global_time += first_arrival
event_list.append(Event('arrival', first_arrival))
event_list.append(Event('departure', global_time + sample_service_time()))
ServiceUnits[0].take_unit()



while custmer_count < N:
    
    current_event = event_list[event_counter]

    # Increment global time
    global_time = current_event.time

    if current_event.event_type == 'arrival':
        custmer_count += 1

        # Check for free service units
        indx, available = check_available_service(ServiceUnits)
        
        if available:
            # Insert departure event and depend to eventlist
            service_time = sample_service_time()
            departure_times.append(service_time)
            
            departure_event = Event('departure', global_time + service_time)
            event_list = apend_event(event_list, departure_event)

            # Take service unit
            ServiceUnits[indx].take_unit()

        if not available:
            # Costumer blocked
            block_count += 1
        
        # insert time for next arrival
        arrival_time = sample_arrival()
        arrival_times.append(arrival_time)
        
        arrival_event = Event('arrival', global_time + arrival_time)
        event_list = apend_event(event_list, arrival_event)

    elif current_event.event_type == 'departure': 
        # Free the service unit for the current departure event
        for unit in ServiceUnits:
            if unit.occupied:
                unit.free_unit()

    # increment event counter
    event_counter += 1
        


print(f"Block probability for single run: {block_count/N}")


In [ ]:
print([event.time for event in event_list])

print(f"Num arrivals: {len(arrival_times)}")
print(f"Num departures: {len(departure_times)}")
print(f"Mean arrival times: {np.mean(arrival_times)}")
print(f"Mean departure times: {np.mean(departure_times)}")

print(f"Blocked count: \n{block_count}\nArrivals - depatures: \n{len(arrival_times) - len(departure_times)}")


